<a href="https://colab.research.google.com/github/MaryamShirazi/GAN/blob/main/MNISTGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# تنظیمات
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
z_dim = 100
lr = 0.0002
epochs = 100

# آماده‌سازی داده
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# معماری شبکه‌ها
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 784),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x.view(-1, 784))

# ایجاد شبکه‌ها و تعریف تابع هزینه و بهینه‌ساز
generator = Generator().to(device)
discriminator = Discriminator().to(device)
criterion = nn.BCELoss()
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = optim.Adam(generator.parameters(), lr=lr)

# تابعی برای آموزش شبکه‌ها
def train_discriminator(images):
    # اطمینان حاصل کنید که اندازه بچ تصاویر با اندازه بچ‌های مورد انتظار برای شبکه‌ها همخوانی دارد
    images = images.view(-1, 28 * 28).to(device)

    # آموزش دهیم که تشخیص دهد که عکس‌ها از دیتاست واقعی یا تولیدی هستند
    real_labels = torch.ones(images.size(0), 1, device=device)
    fake_labels = torch.zeros(images.size(0), 1, device=device)

    # آموزش دادن بر داده‌های واقعی
    outputs = discriminator(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    # آموزش دادن بر داده‌های تولیدی
    z = torch.randn(images.size(0), z_dim, device=device)
    fake_images = generator(z)
    outputs = discriminator(fake_images.detach())
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    # تجمیع تابع هزینه و به‌روزرسانی شبکه
    d_loss = d_loss_real + d_loss_fake
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()

    return d_loss, real_score, fake_score

def train_generator():
    # آموزش دهیم که تشخیص دهد که عکس‌ها واقعی هستند
    real_labels = torch.ones(batch_size, 1, device=device)

    # آموزش دادن بر داده‌های تولیدی و به‌روزرسانی شبکه
    z = torch.randn(batch_size, z_dim, device=device)
    fake_images = generator(z)
    outputs = discriminator(fake_images)
    g_loss = criterion(outputs, real_labels)
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()

    return g_loss, fake_images

# آموزش GAN
d_losses, g_losses, real_scores, fake_scores = [], [], [], []
for epoch in range(epochs):
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)

        # آموزش دهیم تا شبکه‌های تولیدکننده و تشخیص‌دهنده را آموزش دهیم
        d_loss, real_score, fake_score = train_discriminator(images)
        g_loss, fake_images = train_generator()

        # ذخیره کردن و چاپ آمار و پیشرفت
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], '
                  f'D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}, '
                  f'Real Score: {real_score.mean().item():.4f}, Fake Score: {fake_score.mean().item():.4f}')

    # نمونه‌برداری و چاپ نمونه‌های تولید شده در انتهای هر ایپاک
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        fig, axes = plt.subplots(5, 5, figsize=(10,10))
        for i, ax in enumerate(axes.flat):
            ax.imshow(images[i].squeeze().cpu().detach().numpy(), cmap='gray')
            ax.axis('off')
        plt.show()
    elif (epoch+1) % 10 == 0:
        images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        fig, axes = plt.subplots(5, 5, figsize=(10,10))
        for i, ax in enumerate(axes.flat):
            ax.imshow(images[i].squeeze().cpu().detach().numpy(), cmap='gray')
            ax.axis('off')
        plt.show()
